*tool - jupyter notebook (python=3.6)

# 단어의 표현 (Word Representation)
- 단어의 표현: 문자를 수치화하는 방법으로 자연어 처리에 필수
- Local Representation (통계 기반): 단어만을 보고 수치화
    + One-hot Encoding
    + N-gram
    + Count Based (BoW, TDM, TF-IDF)
- Distributed Representation (딥러닝 기반): 주변단어 참고하여 수치화
    + Prediction Based (Word2Vec, FastText)
    + Count Based (Windows-GloVe, Full Document-LSA)

## BOW (Bag of Words)
- 단어의 등장 빈도수에 기반해서 문장(문서)을 표현하는 방법
- 구현 방법
    + 1. 직접 구현하기
    + 2. sklearn : 통째로
    + 3. gensim : 쪼개서
- 단점
    + 단어의 순서 고려하지 않음
    + BoW 는 sparse > 벡터공간의 낭비, 연산 비효율성 초래
    + 단어 빈도수가 무조건 중요도 의미 X > TF-IDF 로 보완
    + 전처리 매우 중요, 같은 의미의 다른 단어 표현 다르게 인식 
        + 뉴스와 같은 매체(+) 소설(-)

## 1. 직접 구현하기
- 공백 토큰화 > 각 토큰에 고유 인덱스 부여 > 각 인덱스 위치에 토큰 등장 횟수 기록

In [1]:
docs = ['오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어',
       '오늘 동물원에서 원숭이에게 사과를 줬어']

In [2]:
# 띄어쓰기 토큰화
doc_ls = []
for doc in docs:
    doc_ls.append(doc.split(' '))
    
doc_ls

[['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '코끼리', '원숭이에게', '먹이를', '줬어'],
 ['오늘', '동물원에서', '원숭이에게', '사과를', '줬어']]

In [3]:
# 빈 딕셔너리 만들어놓기
from collections import defaultdict
word2id = defaultdict(lambda : len(word2id))

word2id

defaultdict(<function __main__.<lambda>()>, {})

In [4]:
# 빈 딕셔너리에 토큰들 하나씩 넣기(중복 방지)
for doc in doc_ls:
    for token in doc:
        word2id[token]       # 고유한 값에만 인덱스 부여(중복 값 추가하지 않음)
        
word2id

defaultdict(<function __main__.<lambda>()>,
            {'오늘': 0,
             '동물원에서': 1,
             '코끼리': 2,
             '원숭이를': 3,
             '보고': 4,
             '원숭이에게': 5,
             '먹이를': 6,
             '줬어': 7,
             '사과를': 8})

In [5]:
import numpy as np
BoW_ls = []
for i, doc in enumerate(doc_ls):
    bow = np.zeros(len(word2id), dtype=int)
    print(bow)
    for token in doc:
        bow[word2id[token]] += 1         # 해당 토큰의 위치(column)
    BoW_ls.append(bow.tolist())
# 1차원 배열    
BoW_ls

[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]


[[1, 1, 2, 1, 1, 1, 1, 1, 0], [1, 1, 0, 0, 0, 1, 0, 1, 1]]

In [6]:
# docs 전체를 하나로 처리하기
import numpy as np
bag_of_words = np.zeros(len(word2id), dtype=int)

for list in doc_ls:
    for word in list:
        index = word2id[word]
        bag_of_words[index] += 1 

bag_of_words

array([2, 2, 2, 1, 1, 2, 1, 2, 1])

In [7]:
# DataFrame화 및 시각화
from IPython.core import display as ICD
import pandas as pd
sorted_vocab = sorted((value, key) for key, value in word2id.items())
print('sorted_vocab', sorted_vocab)

sorted_vocab [(0, '오늘'), (1, '동물원에서'), (2, '코끼리'), (3, '원숭이를'), (4, '보고'), (5, '원숭이에게'), (6, '먹이를'), (7, '줬어'), (8, '사과를')]


In [8]:
vocab = []
for v in sorted_vocab:
    vocab.append(v[1])
print('vocab',vocab)

vocab ['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '원숭이에게', '먹이를', '줬어', '사과를']


In [9]:
for i in range(len(docs)):
    print("문서{} : {}".format(i, docs[1]))
    ICD.display(pd.DataFrame([BoW_ls[i]], columns=vocab))
    print("\n\n")

문서0 : 오늘 동물원에서 원숭이에게 사과를 줬어


,오늘,동물원에서,코끼리,원숭이를,보고,원숭이에게,먹이를,줬어,사과를
0,1,1,2,1,1,1,1,1,0





문서1 : 오늘 동물원에서 원숭이에게 사과를 줬어


,오늘,동물원에서,코끼리,원숭이를,보고,원숭이에게,먹이를,줬어,사과를
0,1,1,0,0,0,1,0,1,1


In [10]:
BoW_ls

[[1, 1, 2, 1, 1, 1, 1, 1, 0], [1, 1, 0, 0, 0, 1, 0, 1, 1]]

## 번외. 단어의 순서를 고려하지 않은 BOW

In [11]:
docs = ['나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해',
       '나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해']

In [12]:
doc_ls = []
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['나는', '양념', '치킨을', '좋아해', '하지만', '후라이드', '치킨을', '싫어해'],
 ['나는', '후라이드', '치킨을', '좋아해', '하지만', '양념', '치킨을', '싫어해']]

In [13]:
from collections import defaultdict
word3id = defaultdict(lambda : len(word3id))

for doc in doc_ls:
    for token in doc:
        word3id[token]
word3id

defaultdict(<function __main__.<lambda>()>,
            {'나는': 0,
             '양념': 1,
             '치킨을': 2,
             '좋아해': 3,
             '하지만': 4,
             '후라이드': 5,
             '싫어해': 6})

In [14]:
import numpy as np
BoW_ls_chi = []
for doc in doc_ls:
    bow = np.zeros(len(word3id), dtype=int)
    print(bow)
    for token in doc:
        bow[word3id[token]] += 1         # 해당 토큰의 위치(column)
    BoW_ls_chi.append(bow.tolist())
# 1차원 배열    
BoW_ls_chi

[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]


[[1, 1, 2, 1, 1, 1, 1], [1, 1, 2, 1, 1, 1, 1]]

In [15]:
word3id.items()

dict_items([('나는', 0), ('양념', 1), ('치킨을', 2), ('좋아해', 3), ('하지만', 4), ('후라이드', 5), ('싫어해', 6)])

In [16]:
from IPython.core import display as ICD
import pandas as pd
sorted_vocab = sorted((value, key) for key, value in word3id.items())
print('sorted_vocab', sorted_vocab)

sorted_vocab [(0, '나는'), (1, '양념'), (2, '치킨을'), (3, '좋아해'), (4, '하지만'), (5, '후라이드'), (6, '싫어해')]


In [17]:
vocab = []
for v in sorted_vocab:
    vocab.append(v[1])
print('vocab',vocab)

vocab ['나는', '양념', '치킨을', '좋아해', '하지만', '후라이드', '싫어해']


In [18]:
for i in range(len(docs)):
    print("문서{} : {}".format(i, docs[1]))
    ICD.display(pd.DataFrame([BoW_ls_chi[i]], columns=vocab))
    print("\n\n")

문서0 : 나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해


,나는,양념,치킨을,좋아해,하지만,후라이드,싫어해
0,1,1,2,1,1,1,1





문서1 : 나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 싫어해


,나는,양념,치킨을,좋아해,하지만,후라이드,싫어해
0,1,1,2,1,1,1,1


## 2. sklearn

In [19]:
docs = ['오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어',
       '오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어 오늘 동물원에서 원숭이에게 사과를 줬어']

In [20]:
# 공백토큰화 > 토큰빈도계산 : CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# 선언
count_vect = CountVectorizer()
BoW = count_vect.fit_transform(docs)  # 전체 문서 넣어주기(리스트 형태로)
print(BoW.toarray())
vocab = count_vect.get_feature_names()   # 각각의 인덱스에 해당하는 이름 알려줌
print(vocab)

[[1 1 1 0 1 1 1 1 2]
 [2 1 1 1 2 1 2 2 2]]
['동물원에서', '먹이를', '보고', '사과를', '오늘', '원숭이를', '원숭이에게', '줬어', '코끼리']


In [21]:
# from IPython.core import display as ICD
import pandas as pd

for i in range(len(docs)):
    print("문서{} : {}".format(i, docs[i]))
    # ICD.display(pd.DataFrame([Bow.toarray()[i], columns=vocab))
    print('\n')
    df = pd.DataFrame([BoW.toarray()[i]],columns=vocab)
    print(df)
    print("\n\n")

문서0 : 오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어


   동물원에서  먹이를  보고  사과를  오늘  원숭이를  원숭이에게  줬어  코끼리
0      1    1   1    0   1     1      1   1    2



문서1 : 오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어 오늘 동물원에서 원숭이에게 사과를 줬어


   동물원에서  먹이를  보고  사과를  오늘  원숭이를  원숭이에게  줬어  코끼리
0      2    1   1    1   2     1      2   2    2





## 3. gensim

In [22]:
# !pip install gensim

In [23]:
docs = ['오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어',
       '오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어 오늘 동물원에서 원숭이에게 사과를 줬어']

In [24]:
# 공백으로 토큰화해서 리스트 생성 > 딕셔너리에 넣기 > 토큰화한 리스트를 doc2bow로 변환
from gensim import corpora

doc_ls = []
for doc in docs:
    doc_ls.append(doc.split())
print(doc_ls)
id2word = corpora.Dictionary(doc_ls)
print('----')
print(id2word)
BoW = []
for doc in doc_ls:
    BoW.append(id2word.doc2bow(doc))
BoW
# 0값을 가지는 아이는 보여주지 않음

[['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '코끼리', '원숭이에게', '먹이를', '줬어'], ['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '코끼리', '원숭이에게', '먹이를', '줬어', '오늘', '동물원에서', '원숭이에게', '사과를', '줬어']]
----
Dictionary(9 unique tokens: ['동물원에서', '먹이를', '보고', '오늘', '원숭이를']...)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)],
 [(0, 2), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 2), (7, 2), (8, 1)]]

In [25]:
BoW[0]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2)]

In [26]:
# DataFrame화 및 시각화
# 보기 편하게 sparse2full 사용(0이어서 안보이는 애를 0으로 채워줌)
from gensim.matutils import sparse2full
from IPython.core import display as ICD
import pandas as pd
vocab = [id2word[i] for i in id2word.keys()]
for i in range(len(docs)):
    print("문서{} : {}".format(i, docs[i]))
    ICD.display(pd.DataFrame([sparse2full(BoW[i], len(vocab))], columns = vocab))
    print("\n\n")

문서0 : 오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어


,동물원에서,먹이를,보고,오늘,원숭이를,원숭이에게,줬어,코끼리,사과를
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0





문서1 : 오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 먹이를 줬어 오늘 동물원에서 원숭이에게 사과를 줬어


,동물원에서,먹이를,보고,오늘,원숭이를,원숭이에게,줬어,코끼리,사과를
0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0


In [27]:
docs = ['오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 사과를 줬어',
       '오늘 동물원에서 코끼리 원숭이를 코끼리 원숭이에게 먹이를 줬어 오늘 동물원에서 원숭이에게 사과를 줬어']

# 공백으로 토큰화해서 리스트 생성 > 딕셔너리에 넣기 > doc2bow로 변환
from gensim import corpora

doc_ls = []
for doc in docs:
    doc_ls.append(doc.split())
print(doc_ls)
id2word = corpora.Dictionary(doc_ls)
print('----')
print(id2word)
BoW = []
for doc in doc_ls:
    BoW.append(id2word.doc2bow(doc))
BoW
# 0값을 가지는 아이는 보여주지 않음

# 보기 편하게 sparse2full 사용(0이어서 안보이는 애를 0으로 채워줌)
from gensim.matutils import sparse2full
from IPython.core import display as ICD
import pandas as pd
vocab = [id2word[i] for i in id2word.keys()]
for i in range(len(docs)):
    print("문서{} : {}".format(i, docs[i]))
    ICD.display(pd.DataFrame([sparse2full(BoW[i], len(vocab))], columns = vocab))
    print("\n\n")

[['오늘', '동물원에서', '코끼리', '원숭이를', '보고', '코끼리', '원숭이에게', '사과를', '줬어'], ['오늘', '동물원에서', '코끼리', '원숭이를', '코끼리', '원숭이에게', '먹이를', '줬어', '오늘', '동물원에서', '원숭이에게', '사과를', '줬어']]
----
Dictionary(9 unique tokens: ['동물원에서', '보고', '사과를', '오늘', '원숭이를']...)
문서0 : 오늘 동물원에서 코끼리 원숭이를 보고 코끼리 원숭이에게 사과를 줬어


,동물원에서,보고,사과를,오늘,원숭이를,원숭이에게,줬어,코끼리,먹이를
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0





문서1 : 오늘 동물원에서 코끼리 원숭이를 코끼리 원숭이에게 먹이를 줬어 오늘 동물원에서 원숭이에게 사과를 줬어


,동물원에서,보고,사과를,오늘,원숭이를,원숭이에게,줬어,코끼리,먹이를
0,2.0,0.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0


In [28]:
BoW[1]

[(0, 2), (2, 1), (3, 2), (4, 1), (5, 2), (6, 2), (7, 2), (8, 1)]